In [1]:
import numpy as np
import cv2
import yaml

In [ ]:
# --- 1. Größe der Checkerboard‑Kästchen festlegen (z. B. 30 mm) ------------
square_size = 34.0        # Größe eines einzelnen Kästchens in mm

# --- 2. Terminierungskriterien ------------------------------------------------
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# --- 3. Objekt‑Punkte vorbereiten (real‑world 3D Koordinaten) ----------------
# 6 × 7 Checkerboard (7 Felder pro Zeile, 6 Felder pro Spalte)
objp = np.zeros((6*7, 3), np.float32)

# Erzeuge die (x, y) Koordinaten der Felder und skaliere sie mit square_size
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2) * square_size

# --- 4. Arrays für Objekt‑ und Bild‑Punkte -----------------------------------
objpoints = []  # 3D Punkte im realen Raum
imgpoints = []  # 2D Punkte im Bild

In [4]:
cap = cv2.VideoCapture(0)
found = 0
while(found < 10):  # Here, 10 can be changed to whatever number you like to choose
    ret, img = cap.read() # Capture frame-by-frame
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)   # Certainly, every loop objp is the same, in 3D.
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2, ret)
        found += 1
        print(f'Found: {found}')

    cv2.imshow('img', img)
    cv2.waitKey(10)

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/lbraun/Robotik/3. Semester/Robotik Projekt 2/MURI/misc/fuck_around_and_find_out/venv/lib/python3.12/site-packages/cv2/qt/plugins"


Found: 1
Found: 2
Found: 3
Found: 4
Found: 5
Found: 6
Found: 7
Found: 8
Found: 9
Found: 10


In [5]:
# It's very important to transform the matrix to list.

data = {'camera_matrix': np.asarray(mtx).tolist(), 'dist_coeff': np.asarray(dist).tolist()}

with open("calibration.yaml", "w") as f:
    yaml.dump(data, f)